# 1. Install and import packages

* 본 실습에 필요한 패키지를 설치합니다.
* KorQuAD 에 사전학습된 모델을 사용하기 위하여 KoBERT-KorQuAD 를 설치합니다.

In [1]:
!git clone https://github.com/monologg/KoBERT-KorQuAD kobert
!pip install transformers tqdm sentencepiece

Cloning into 'kobert'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 110 (delta 63), reused 69 (delta 30), pack-reused 0
Receiving objects: 100% (110/110), 7.60 MiB | 12.45 MiB/s, done.
Resolving deltas: 100% (63/63), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00


In [2]:
import os, json
import numpy as np
from tqdm.notebook import tqdm

import argparse
import glob
import logging
import os
import random
import timeit

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers.data.metrics.squad_metrics import (
    compute_predictions_log_probs,
    compute_predictions_logits,
    squad_evaluate,
)
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    BertConfig,
    BertForQuestionAnswering,
    BertTokenizer,
    get_linear_schedule_with_warmup,
    squad_convert_examples_to_features
)

# 2. Explore Data

## 2.1. Raw data 확인

In [3]:
train_data = json.load(open('./kobert/data/KorQuAD_v1.0_train.json', 'r'))
dev_data = json.load(open('./kobert/data/KorQuAD_v1.0_dev.json', 'r'))

In [4]:
train_data["data"] = train_data["data"][:150]
with open('./kobert/data/KorQuAD_v1.0_train.json', 'w') as fout:
  json.dump(train_data, fout, indent=2)

dev_data["data"] = dev_data["data"][:50]
with open('./kobert/data/KorQuAD_v1.0_dev.json', 'w') as fout:
  json.dump(dev_data, fout, indent=2)

In [5]:
# train_data["data"]
print("Nb of data: ", len(train_data["data"]))
print()
print(train_data["data"][0].keys())
print(len(train_data["data"][0]["paragraphs"]))
print("QA example: ")
for k, v in train_data["data"][0]["paragraphs"][0]["qas"][0].items():
    print(k, v)
print()
print("Context example: ")
train_data["data"][0]["paragraphs"][0]["context"]

Nb of data:  150

dict_keys(['paragraphs', 'title'])
3
QA example: 
answers [{'text': '교향곡', 'answer_start': 54}]
id 6566495-0-0
question 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?

Context example: 


'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

## 2.2. 모델에 입력될 데이터로 변환

* 한 데이터당 context, qa pair 가 tokenizer 로 처리되어 모델에 입력되도록 전처리를 해야합니다.
* 전처리를 위하여 transformers 라이브러리에서 제공하는 함수를 사용합니다.
* https://huggingface.co/monologg/kobert




In [6]:
from transformers.data.processors.squad import SquadProcessor, SquadV1Processor
from transformers.data.processors.squad import squad_convert_examples_to_features

In [7]:
processor = SquadV1Processor()
train_examples = processor.get_train_examples('./kobert/data', 'KorQuAD_v1.0_train.json')

100%|██████████| 150/150 [00:03<00:00, 46.62it/s]


In [8]:
from transformers import BertModel, BertConfig, AdamW
from kobert.tokenization_kobert import KoBertTokenizer

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [9]:
print(train_examples[0].__dict__.keys())

dict_keys(['qas_id', 'question_text', 'context_text', 'answer_text', 'title', 'is_impossible', 'answers', 'start_position', 'end_position', 'doc_tokens', 'char_to_word_offset'])


In [10]:
print(train_examples[0].question_text)

바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [11]:
features, train_dataset = squad_convert_examples_to_features(
            examples=train_examples,
            tokenizer=tokenizer, # Tokenizer 설정
            max_seq_length=512,
            doc_stride=128, 
            max_query_length=64,
            is_training=True,
            return_dataset="pt", 
            tqdm_enabled=False)

In [12]:
print(features[0].__dict__.keys())

dict_keys(['input_ids', 'attention_mask', 'token_type_ids', 'cls_index', 'p_mask', 'example_index', 'unique_id', 'paragraph_len', 'token_is_max_context', 'tokens', 'token_to_orig_map', 'start_position', 'end_position', 'is_impossible', 'qas_id', 'encoding'])


In [13]:
print(features[0].input_ids)

[2, 2186, 5538, 5691, 5760, 1101, 7618, 7095, 4799, 7005, 6691, 6116, 3824, 5439, 2095, 6884, 7088, 3084, 5439, 7147, 5017, 5760, 5330, 258, 3, 547, 155, 5712, 2186, 5538, 5691, 5760, 1101, 7618, 7095, 4799, 7005, 6691, 7088, 4468, 3824, 5439, 1185, 1449, 6896, 1917, 7096, 517, 5644, 6060, 3686, 2846, 6079, 5007, 4928, 7095, 1103, 7886, 5443, 7088, 3084, 6062, 1873, 825, 5760, 5782, 54, 3647, 2965, 2186, 5538, 5691, 5760, 547, 154, 5719, 2553, 1725, 7448, 7078, 2640, 7207, 6629, 7213, 1562, 517, 0, 7086, 2693, 6003, 4211, 7217, 5468, 3039, 6896, 745, 7877, 2016, 7768, 7628, 7717, 6053, 6116, 1933, 5760, 4799, 7005, 6691, 7095, 3063, 6896, 1025, 7870, 4965, 54, 1864, 4799, 6122, 6903, 3093, 6432, 5702, 7565, 7095, 4348, 6079, 4799, 6122, 3610, 7020, 1073, 7903, 6811, 5788, 7096, 3332, 7276, 7794, 2333, 7628, 6347, 7095, 1103, 7886, 5443, 627, 6328, 7088, 1800, 1338, 784, 6209, 2229, 5761, 46, 3650, 7096, 3647, 5941, 7848, 529, 7034, 4799, 7005, 6691, 7095, 2718, 5443, 7078, 3084, 6916, 

In [14]:
# See what's inside a sample in dataset
# From documentation:
#  0 all_input_ids,
#  1 all_attention_masks,
#  2 all_token_type_ids,
#  3 all_start_positions,
#  4 all_end_positions,
#  5 all_cls_index,
#  6 all_p_mask,
#  7 all_is_impossible,
for i, item in enumerate(train_dataset[200]):
  print(i)
  print(item)

0
tensor([   2, 2048, 3133, 7673, 5756, 7263, 7095, 1023, 7636, 7206, 3890, 7202,
        2781, 7350, 7086,  258,    3, 3133, 7673, 5756, 7263, 7086, 4063, 7206,
         517, 5552, 7100,   54, 2048, 1096, 6241, 7096, 2423, 5920, 6044, 5439,
         517, 7086, 6458, 7088,  517, 5999, 5782,   54,   18, 5538, 6037, 5655,
        1023, 5561, 4257, 6904, 2339, 6538, 5900,   40,  774, 4688, 2181, 5859,
        6983,  517, 5330, 6607, 6577, 4193, 7831,   54, 3811, 5760, 4642, 6079,
        3886, 6116, 2874, 3862,   54, 2514, 7199, 7846, 7086,  543, 6903,  544,
        2618, 7095,  517,  489,  329,  401,  423,  478,  384, 7100,   54, 4737,
        6111, 7095,  953, 5859, 5760, 1923, 2423, 5920, 6037, 7010,  807, 7427,
        5468, 2533, 7798,   54,  820,  517, 5330, 6940, 5899, 2912, 6629, 7828,
        4737, 6111, 7086, 4197, 7096, 7417, 1168, 6135, 2872, 3854, 5378, 4473,
        6493, 6079, 1940, 2874, 3862,   54, 4737, 6111, 7086, 2181, 5859, 5330,
        3498, 6003, 5756, 5468,  517, 

In [15]:
" ".join(tokenizer.convert_ids_to_tokens(train_dataset[200][0]))

'[CLS] ▁모든 ▁악 티 늄 족 의 ▁공 통 적인 ▁자기 적 ▁성 질 은 ? [SEP] ▁악 티 늄 족 은 ▁전형 적인 ▁ 금속 이다 . ▁모든 ▁광 물 이 ▁부 드 럽 고 ▁ 은 빛 을 ▁ 띈 다 . ( 그 러 나 ▁공 기 ▁중 에서는 ▁변 색 된다 ) ▁각각 ▁큰 ▁밀 도 와 ▁ 가 소 성이 ▁존재 한다 . ▁일부 는 ▁칼 로 ▁자 를 ▁수도 ▁있다 . ▁비 저 항 은 ▁15 에서 ▁150 ▁사이 의 ▁ μ O h m · cm 이다 . ▁토 륨 의 ▁경 도 는 ▁마치 ▁부 드 러 운 ▁강 철 과 ▁비슷 하다 . ▁강하게 ▁ 가 열 된 ▁순 수 한 ▁토 륨 은 ▁종 이 처럼 ▁굴 릴 ▁수 ▁있 거나 ▁철 사 로 ▁만들 ▁수도 ▁있다 . ▁토 륨 은 ▁밀 도 가 ▁우 라 늄 과 ▁ 플 루 토 늄 의 ▁절반 ▁정도 ▁된다 . ▁그러나 ▁우 라 늄 과 ▁ 플 루 토 늄 보다 ▁단 단 하다 . ▁모든 ▁악 티 늄 족 은 ▁방 사 성 ▁원 소 이며 , ▁상 자 성을 ▁ 띄 고 , ▁이례적 인 ▁경우 에는 , ▁여러 ▁결정 질 ▁상 을 ▁가지고 ▁있다 . ▁ 플 루 토 늄 은 ▁7 개 , ▁그리고 ▁우 라 늄 , ▁ [UNK] 투 늄 ▁그리고 ▁ 캘 리 포 늄 은 ▁3 개 이다 . ▁프로 탁 티 늄 , ▁우 라 늄 , ▁ [UNK] 투 늄 , ▁그리고 ▁ 플 루 토 늄 의 ▁결정 ▁구조 는 ▁ 란 타 넘 족 과 ▁비슷 하게 ▁깨끗 하지 ▁않은 ▁상 사 형 이 고 ▁3 d 의 ▁형태 를 ▁가지고 ▁있어 ▁전 이 ▁ 금속 과 ▁비슷 하다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [16]:
print(train_dataset[200][3], train_dataset[200][4])
" ".join(tokenizer.convert_ids_to_tokens(train_dataset[200][0][train_dataset[200][3]:train_dataset[200][4]+1]))

tensor(186) tensor(188)


'▁상 자 성을'

# 3. Load model
* 모델은 사전학습된 KoBERT-KorQuAD 를 사용합니다.
* 모델의 from_pretrained 인자에 들어갈 model shortcut 에 따라 가져오는 pretrained model 이 달라집니다.
* https://huggingface.co/monologg/kobert


In [17]:
####################### TODO ###############################
# model, tokenizer, config 를 선언하세요. 
# model shortcut: monologg/kobert
# BertForQuestionAnswering, KoBertTokenizer, BertConfig
##############################################################
model = BertForQuestionAnswering.from_pretrained("monologg/kobert")
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
config = BertConfig.from_pretrained("monologg/kobert")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [18]:
# model 을 cuda 에 올리기
device = torch.device("cuda" )
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

# 4. Train the model 

## 4.1. Training parameter setup

In [19]:
# training 을 위한 파라미터 설정
params = {
 'max_seq_length': 512,
 'doc_stride': 128,
 'max_query_length': 64,
 'do_lower_case': False,

 'num_train_epochs': 2,
 'per_gpu_train_batch_size': 8,
 'per_gpu_eval_batch_size': 8,

 'learning_rate': 5e-05,
 'gradient_accumulation_steps': 2,
 'weight_decay': 0.0,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'warmup_steps': 0,
 'log_steps': 30,
 'save_steps': 200,
 'output_dir': 'models',
 'max_answer_length': 30,
 'n_best_size': 20,
 'threads': 1 
}

In [20]:
from argparse import Namespace
args = argparse.Namespace()
args = vars(args)

args.update(params)
args = Namespace(**args)

## 4.2. 데이터 준비

In [21]:
train_dataloader = DataLoader(train_dataset, batch_size=args.per_gpu_train_batch_size, shuffle=True)

## 4.3. Start training!

In [22]:
# Optimizer 설정
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, eps=args.adam_epsilon)

In [23]:
# Scheduler 사용할 경우
# optimizer = AdamW(model.parameters(), lr=args.learning_rate, eps=args.adam_epsilon)
# t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
# scheduler = get_linear_schedule_with_warmup(
#     optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
# )

In [24]:
global_step = 1

tr_loss, logging_loss = 0.0, 0.0
model.zero_grad()
model.train()
for _ in range(args.num_train_epochs):
    epoch_iterator = tqdm(train_dataloader, desc="Iteration", position=0)
    for step, batch in enumerate(epoch_iterator):      
        batch = tuple(t.to(device) for t in batch)

        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
            "start_positions": batch[3],
            "end_positions": batch[4],
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss = loss / args.gradient_accumulation_steps
        loss.backward()

        tr_loss += loss.item()
        global_step += 1

        if (step+1) % args.gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
            optimizer.step()
            # scheduler.step() 
            model.zero_grad()

        if global_step % args.log_steps == 0:  
          print('Global Step: {} | Train Loss: {:.3f} '.format(global_step, (tr_loss-logging_loss)/args.log_steps))
          logging_loss = tr_loss
            
        if args.save_steps > 0 and global_step % args.save_steps == 0:
            if not os.path.exists(args.output_dir):
                os.makedirs(args.output_dir)
            torch.save(model.state_dict(), os.path.join(args.output_dir, "checkpoint{}.pth".format(global_step)))
            print('Model saved!')

Iteration:   0%|          | 0/862 [00:00<?, ?it/s]

Global Step: 30 | Train Loss: 2.748 
Global Step: 60 | Train Loss: 2.375 
Global Step: 90 | Train Loss: 2.200 
Global Step: 120 | Train Loss: 2.133 
Global Step: 150 | Train Loss: 2.054 
Global Step: 180 | Train Loss: 1.850 
Model saved!
Global Step: 210 | Train Loss: 1.729 
Global Step: 240 | Train Loss: 1.577 
Global Step: 270 | Train Loss: 1.526 
Global Step: 300 | Train Loss: 1.453 
Global Step: 330 | Train Loss: 1.444 
Global Step: 360 | Train Loss: 1.380 
Global Step: 390 | Train Loss: 1.337 
Model saved!
Global Step: 420 | Train Loss: 1.316 
Global Step: 450 | Train Loss: 1.244 
Global Step: 480 | Train Loss: 1.242 
Global Step: 510 | Train Loss: 1.303 
Global Step: 540 | Train Loss: 1.223 
Global Step: 570 | Train Loss: 1.222 
Global Step: 600 | Train Loss: 1.166 
Model saved!
Global Step: 630 | Train Loss: 1.250 
Global Step: 660 | Train Loss: 1.160 
Global Step: 690 | Train Loss: 1.241 
Global Step: 720 | Train Loss: 1.271 
Global Step: 750 | Train Loss: 1.126 
Global Step: 7

Iteration:   0%|          | 0/862 [00:00<?, ?it/s]

Global Step: 870 | Train Loss: 0.980 
Global Step: 900 | Train Loss: 0.888 
Global Step: 930 | Train Loss: 0.957 
Global Step: 960 | Train Loss: 0.918 
Global Step: 990 | Train Loss: 0.908 
Model saved!
Global Step: 1020 | Train Loss: 0.902 
Global Step: 1050 | Train Loss: 0.920 
Global Step: 1080 | Train Loss: 0.912 
Global Step: 1110 | Train Loss: 0.790 
Global Step: 1140 | Train Loss: 0.879 
Global Step: 1170 | Train Loss: 0.861 
Global Step: 1200 | Train Loss: 0.937 
Model saved!
Global Step: 1230 | Train Loss: 0.869 
Global Step: 1260 | Train Loss: 1.036 
Global Step: 1290 | Train Loss: 0.900 
Global Step: 1320 | Train Loss: 0.823 
Global Step: 1350 | Train Loss: 0.827 
Global Step: 1380 | Train Loss: 0.963 
Model saved!
Global Step: 1410 | Train Loss: 0.892 
Global Step: 1440 | Train Loss: 0.831 
Global Step: 1470 | Train Loss: 0.938 
Global Step: 1500 | Train Loss: 0.884 
Global Step: 1530 | Train Loss: 0.910 
Global Step: 1560 | Train Loss: 0.939 
Global Step: 1590 | Train Loss

# 5. Evaluate

## 5.1. Load eval dataset

In [25]:
eval_examples = processor.get_dev_examples('./kobert/data', 'KorQuAD_v1.0_dev.json')

100%|██████████| 50/50 [00:00<00:00, 165.72it/s]


In [26]:
eval_features, eval_dataset = squad_convert_examples_to_features(
            examples=eval_examples,
            tokenizer=tokenizer,
            max_seq_length=args.max_seq_length,
            doc_stride=args.doc_stride,
            max_query_length=args.max_query_length,
            is_training=False,
            return_dataset="pt",
            tqdm_enabled=False,
            threads=args.threads,
        )

In [27]:
args.eval_batch_size = args.per_gpu_eval_batch_size
eval_sampler = SequentialSampler(eval_dataset)
eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, 
                             batch_size=args.eval_batch_size)

## 5.2. Evaluate

In [28]:
all_results = []

model.eval()
for batch in tqdm(eval_dataloader, desc="Evaluating", position=0, leave=True):    
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
        }

        example_indices = batch[3]

        outputs = model(**inputs)
           
    for i, example_index in enumerate(example_indices):
        eval_feature = eval_features[example_index.item()]
        unique_id = int(eval_feature.unique_id)

        start_logits = outputs.start_logits[i].detach().cpu().tolist()
        end_logits = outputs.end_logits[i].detach().cpu().tolist()
        
        result = SquadResult(unique_id, start_logits, end_logits)

        all_results.append(result)

Evaluating:   0%|          | 0/212 [00:00<?, ?it/s]

In [29]:
output_prediction_file = os.path.join("predictions.json")

In [30]:
predictions = compute_predictions_logits(
    eval_examples,
    eval_features,
    all_results,
    args.n_best_size,
    args.max_answer_length,
    args.do_lower_case,
    output_prediction_file,
    output_nbest_file=None,
    output_null_log_odds_file=None,
    verbose_logging=False,
    version_2_with_negative=False,
    null_score_diff_threshold=0.0,
    tokenizer=tokenizer
)

In [31]:
results = squad_evaluate(eval_examples, predictions)

In [32]:
results

OrderedDict([('exact', 36.86967910936477),
             ('f1', 44.95671297127433),
             ('total', 1527),
             ('HasAns_exact', 36.86967910936477),
             ('HasAns_f1', 44.95671297127433),
             ('HasAns_total', 1527),
             ('best_exact', 36.86967910936477),
             ('best_exact_thresh', 0.0),
             ('best_f1', 44.95671297127433),
             ('best_f1_thresh', 0.0)])